### Joining transformed dataset from Michelle and Ming with Beneficiary Information
##### Adding following columns:
- BENE_SEX_IDENT_CD -  Male=0, Female=1
- BENE_RACE_CD - orginal categories from file, 1:'White', 2:'Black', 3:'Other', 5:'Hispanic'
- SP_STATE_CODE - coded by region South (S), Northeast (N), Midwest (M), West (W), Other (O)
- Age

In [1]:
# import and read file, use NaN for elements with no values
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

df = pd.read_csv("OpToIp_OpNoAdmsn_Merged_Reduced_agg_DiagCode.csv", na_values = ['no info', '.'])
print(df.head(5))

          MEMBER_ID  1  2  3  4  5  6  7  8  9    ...     12  13  14  15  16  \
0  00000B48BCF4AD29  0  1  1  0  0  0  0  0  0    ...      0   5   0   0   0   
1  00001C24EE7B06AC  0  0  0  0  0  0  1  0  0    ...      0   0   1   0   1   
2  00003539A5D77654  0  0  0  0  2  0  1  0  0    ...      0   0   0   0   0   
3  0000525AB30E4DEF  0  0  2  0  0  1  1  0  1    ...      5   0   0   0   2   
4  00005B007DDAAC31  0  0  0  0  0  0  1  0  0    ...      0   0   0   0   1   

   17  18  19  Median(days_op_to_ip)  HF label  
0   0   1   0                   93.0         1  
1   0   2   0                    0.0         0  
2   0   0   0                    0.0         0  
3   0   1   0                   18.5         0  
4   0   4   0                    0.0         0  

[5 rows x 22 columns]


In [2]:
# read in beneficiary file so can merge some features from file (age, race, sex,state)
df_merge = pd.read_csv("combined_ben_sum_AddYear.csv", na_values = ['no info', '.'])
#subset of data to merge
df_merge_subset = df_merge[['DESYNPUF_ID','Year','BENE_BIRTH_DT','BENE_SEX_IDENT_CD','BENE_RACE_CD','SP_STATE_CODE']]
#calculate age (year column - year from BENE_BIRTH_DT)
#newdf["Birth_Year"] = pd.to_datetime(newdf['BENE_BIRTH_DT']).dt.year
df_merge_subset.loc[:,'Age'] = df_merge_subset.loc[:,'Year'] - (pd.to_datetime(df_merge_subset.loc[:,'BENE_BIRTH_DT'],
                                                                             format='%Y%m%d').dt.year)

C:\Users\barba\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\barba\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [3]:
# 0 - Male, 1 - Female
#df_merge_subset.loc[:,'BENE_RACE_CD'] = df_merge_subset.loc[:,'BENE_RACE_CD'].replace({1:'White',2:'Black',3:'Other', 5:'Hispanic'}).apply(str)
df_merge_subset.loc[:,'BENE_SEX_IDENT_CD'] = df_merge_subset.loc[:,'BENE_SEX_IDENT_CD'].replace({1:0,2:1})
# converting number code to state name
df_merge_subset.loc[:,'SP_STATE_CODE'] = df_merge_subset.loc[:,'SP_STATE_CODE'].replace({
    1:'AL',2:'AK',3:'AZ',4:'AR',5:'CA',6:'CO',7:'CT',8:'DE',9:'DC',10:'FL',11:'GA',12:'HI',
    13:'ID',14:'IL',15:'IN',16:'IA',17:'KS',18:'KY',19:'LA',20:'ME',21:'MD',22:'MA',23:'MI',
    24:'MN',25:'MS',26:'MO',27:'MT',28:'NE',29:'NV',30:'NH',31:'NJ',32:'NM',33:'NY',34:'NC',
    35:'ND',36:'OH',37:'OK',38:'OR',39:'PA',41:'RI',42:'SC',43:'SD',44:'TN',45:'TX',
    46:'UT',47:'VT',49:'VA',50:'WA',51:'WV',52:'WI',53:'WY',54:'Other'})
# convert state to regions (South (S), Northeast (N), Midwest (M), West (W), Other (O))
df_merge_subset.loc[:,'SP_STATE_CODE'] = df_merge_subset.loc[:,'SP_STATE_CODE'].replace({
       'AK': 'O', 'AL': 'S','AR': 'S','AZ': 'W','CA': 'W','CO': 'W','CT': 'N','DC': 'N','DE': 'N',
        'FL': 'S','GA': 'S','HI': 'O','IA': 'M','ID': 'W','IL': 'M','IN': 'M','KS': 'M','KY': 'S',
        'LA': 'S','MA': 'N','MD': 'N','ME': 'N','MI': 'W','MN': 'M','MO': 'M', 'MS': 'S','MT': 'W',
        'NC': 'S','ND': 'M','NE': 'W','NH': 'N','NJ': 'N','NM': 'W','NV': 'W','NY': 'N','OH': 'M',
        'OK': 'S','OR': 'W','PA': 'N','RI': 'N','SC': 'S','SD': 'M','TN': 'S','TX': 'S','UT': 'W',
        'VA': 'S','VT': 'N','WA': 'W','WI': 'M','WV': 'S','WY': 'W', 'Other': 'O'})
print(df_merge_subset.head(5))

        DESYNPUF_ID  Year  BENE_BIRTH_DT  BENE_SEX_IDENT_CD  BENE_RACE_CD  \
0  00013D2EFD8E45D1  2008       19230501                  0             1   
1  00016F745862898F  2008       19430101                  0             1   
2  0001FDD721E223DC  2008       19360901                  1             1   
3  00021CA6FF03E670  2008       19410601                  0             5   
4  00024B3D2352D2D0  2008       19360801                  0             1   

  SP_STATE_CODE  Age  
0             M   85  
1             N   65  
2             N   72  
3             W   67  
4             M   72  


In [4]:
#also delete duplicate DESYNPUF_ID before merge
df_merge_subset = df_merge_subset.drop_duplicates(['DESYNPUF_ID'], keep='first')

In [5]:
# join df_merge_subset with original df so will now have additial features to work with such as age, race, sex, state
# first rename columns to merge on so match
df_merge_subset.rename(columns={'DESYNPUF_ID':'MEMBER_ID'}, inplace = True)
# now merge
df_join = pd.merge(df,
                 df_merge_subset[['MEMBER_ID', 'BENE_SEX_IDENT_CD','SP_STATE_CODE','BENE_RACE_CD','Age']],
                 on='MEMBER_ID')

In [10]:
print(df_join.head(50))

           MEMBER_ID  1  2   3  4  5  6   7  8  9 ...   16  17  18  19  \
0   00000B48BCF4AD29  0  1   1  0  0  0   0  0  0 ...    0   0   1   0   
1   00001C24EE7B06AC  0  0   0  0  0  0   1  0  0 ...    1   0   2   0   
2   00003539A5D77654  0  0   0  0  2  0   1  0  0 ...    0   0   0   0   
3   0000525AB30E4DEF  0  0   2  0  0  1   1  0  1 ...    2   0   1   0   
4   00005B007DDAAC31  0  0   0  0  0  0   1  0  0 ...    1   0   4   0   
5   000064755F16C901  0  1   1  2  2  2   4  0  2 ...    2   1   6   0   
6   00006BCF17C9043D  0  0   8  2  3  2   6  3  2 ...    8   0   7   0   
7   000072CF62193213  0  4   0  0  1  2   7  3  1 ...    7   1   8   0   
8   0000838E2BBC2ADA  0  0   1  0  0  0   0  0  0 ...    0   0   0   0   
9   00008A3579F4E065  0  2  12  0  7  2  16  2  0 ...   12   0  15   0   
10  00009465E4889B32  0  0   3  0  0  0   0  0  0 ...    0   1   3   0   
11  00009C897C3D8372  0  0   1  0  0  0   0  0  0 ...    1   0   0   0   
12  0000A38A6DB6581C  0  3   0  0  0  

In [12]:
df_join.to_csv(r'C:\Users\barba\Documents\Jupyter Notebooks\healthcare_analytics_medicare_DESynPUF_dataset\OpToIp_OpNoAdmsn_Merged_Reduced_agg_DiagCode_join_BenefTable.csv')